In [1]:
#importing packages
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as k

#the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [2]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [4]:
hist=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
print("the model has successfully trained")
model.save('mnist.hs')
print("saving the model as mnist.hs")

Epoch 1/10
469/469 [==============================] - 46s 97ms/step - loss: 2.2686 - accuracy: 0.1784 - val_loss: 2.2157 - val_accuracy: 0.4108
Epoch 2/10
469/469 [==============================] - 49s 104ms/step - loss: 2.1635 - accuracy: 0.4053 - val_loss: 2.0924 - val_accuracy: 0.6908
Epoch 3/10
469/469 [==============================] - 48s 103ms/step - loss: 2.0195 - accuracy: 0.5564 - val_loss: 1.9117 - val_accuracy: 0.7554
Epoch 4/10
469/469 [==============================] - 49s 104ms/step - loss: 1.8148 - accuracy: 0.6296 - val_loss: 1.6555 - val_accuracy: 0.7758
Epoch 5/10
469/469 [==============================] - 51s 108ms/step - loss: 1.5517 - accuracy: 0.6726 - val_loss: 1.3496 - val_accuracy: 0.7931
Epoch 6/10
469/469 [==============================] - 49s 105ms/step - loss: 1.2825 - accuracy: 0.7067 - val_loss: 1.0708 - val_accuracy: 0.8156
Epoch 7/10
469/469 [==============================] - 50s 106ms/step - loss: 1.0656 - accuracy: 0.7349 - val_loss: 0.8657 - val_acc

INFO:tensorflow:Assets written to: mnist.hs\assets


INFO:tensorflow:Assets written to: mnist.hs\assets


saving the model as mnist.hs


In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ',score[0])
print('Test accuary: ',score[1])

Test loss:  0.5685045123100281
Test accuary:  0.866599977016449


In [6]:
import tkinter
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [8]:
model = load_model('mnist.hs')
def predict_digit(img):
  #resize image to 28x28 pixels
  img = img.resize((28,28))
  #convert rgb to grayscale
  img = img.convert('L')
  img =  np.array(img)
  #reshaping to support our model input and normalizing
  img = img.reshape(1, 28, 28, 1)
  img = img/255.0
  #predicting the class
  res = model.predict([img])[0]
  return np.argmax(res),max(res)
class App(tk.Tk):
  def __init__(self):
    tk.Tk.__init__(self)
    self.x = self.y = 0
    #creating elements
    self.canvas = tk.Canvas(self,width=300, height=300,bg = "white", cursor= "cross")
    self.label = tk.Label(self, text="Thinking..", font=("Helvetica",48))
    self.classify_btn = tk.Button(self, text = "Recognize", command = self.classify_handwriting)
    self.button_clear = tk.Button(self, text = "clear", command = self.clear_all)
    #Grid structure
    self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
    self.label.grid(row=0, column=1, pady=2, padx=2)
    self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
    self.button_clear.grid(row=1, column=0, pady=2)
    #self.canvas.bind("<Motion>", self.start_pos)
    self.canvas.bind("<B1-Motion>", self.draw_lines)
  def clear_all(self):
    self.canvas.delete("all")
  def classify_handwriting(self):
    HWND = self.canvas.winfo_id() #get the handle of the canvas
    rect = win32gui.GetWindowRect(HWND) #get the coordinate of the canvas
    im = ImageGrab.grab(rect)
    digit, acc = predict_digit(im)
    self.label.configure(text= str(digit)+', '+str(int(acc*100))+'%')
  def draw_lines(self, event):
    self.x = event.x
    self.y = event.y
    r=8
    self.canvas.create_oval(self.x-r,self.y-r,self.x+r, self.y+r)
app = App()
mainloop()

1/1 [==============================] - 0s 20ms/step
